In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
from qiskit.quantum_info import Pauli
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute
from qiskit import Aer
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator,Z2Symmetries
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy
import time
import numba
from numba import jit

In [2]:
def countYgates(pauli_label):
    countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
    return countYgates
def qubitOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('parity')
    return qubit_op
def egBandHamiltonian(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites]             = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    qubitH=Z2Symmetries.two_qubit_reduction(qubitH, 4)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
    w,v=numpy.linalg.eigh(Hmat)
    Eg=w[0]
    state_g=v[:,0]
    return qubitH,Hmat,Eg,state_g

In [3]:
def commutator(A,B):
        B2=WeightedPauliOperator([[-B.paulis[0][0],B.paulis[0][1]]])
        return A.multiply(B).add(B2.multiply(A))
def commutatorsForGradient(Op):
    Op_conj=WeightedPauliOperator([[-1j,Op.paulis[0][1]]])
    M=Op.multiply(qubitH).add(qubitH.multiply(Op_conj))
    Mmat=op_converter.to_matrix_operator(M).matrix
    return Mmat
def commutatorPool(qubitH,stripZs=False):
    def stripZ(label):
        label=numpy.array(list(label))
        label[label=='Z']='I'
        label=''.join(list(label))
        return label
    #construct commutator pool from the Hamiltonian
    pool_H=[WeightedPauliOperator([[1j,qubitH.paulis[i][1]]]) for i in range(len(qubitH.paulis))][1:]
    #commutator between operators
    commutator_pool=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in pool_H for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1]
    labels=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(commutator_pool))]
    unique_labels=numpy.unique(labels)
    if stripZs==True:
        new_labels=[]
        for i in range(len(unique_labels)):
            new_labels.append(stripZ(unique_labels[i]))
        unique_labels=numpy.unique(new_labels)    
    commutator_pool=[WeightedPauliOperator([[1j,Pauli(unique_labels[i])]]) for i in range(len(unique_labels))]
    print(len(commutator_pool))
    return commutator_pool
def compute_gradient(Op,state):
    Op_mat=op_converter.to_matrix_operator(Op).dense_matrix
    grad=numpy.abs(state_g@Op_mat@numpy.conjugate(state))**2
    return grad
def SMO(cost,params,runs=20,tol=1e-4,save_opt_steps=False):
    index=1
    conv_err=1000
    def E_landscape(ind,ang,cost,params):
        params1=copy.deepcopy(params)
        params1[ind]=params1[ind]+ang #ang
        #circ=var_form_base.construct_circuit(parameters=params1)
        E=cost(params1)
        return E.real
    def determine_unknowns(E,cost,params,ind):
        L1=E#_landscape(ind,0,params,Hmat)
        L2=E_landscape(ind,numpy.pi/4.,cost,params)
        L3=E_landscape(ind,-numpy.pi/4.,cost,params)
        ratio=(L3-L2)/(2*L1-L2-L3)
        a3=(L2+L3)/2.
        a2=2*params[ind]-numpy.arctan(ratio)
        a1=(L1-a3)/numpy.cos(numpy.arctan(ratio))
        return a1,a2,a3
    def update(E,cost,params,ind):
        a1,a2,a3=determine_unknowns(E,cost,params,ind)
        thetaStar=a2/2.+numpy.pi/2. if a1>0 else a2/2.
        newParams=copy.deepcopy(params)
        newParams[ind]=thetaStar
        updEnergy=a3-a1 if a1>0 else a3+a1
        return newParams,updEnergy.real
    while conv_err>tol and index<runs:
        print("looped "+str(index)+" times")
        Eold=cost(params)
        init=0
        for i in range(len(params)):#[::-1][0:1]:
            #first run sequential minimial optimization (SMO)  for a given multiqubit operator using 
            #exact analytical form for cost function
            if init==0:
                E=Eold
            ind=i
            params,E=update(E,cost,params,ind)
            if save_opt_steps==True:
                with open('paramsForQubitAdapt_eg_model_parity_fidelity.txt','a') as f:
                    Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
                    print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
            else:
                continue
            init=init+1  
        conv_err=Eold-E
        print("inner loop error",conv_err)
        index=index+1
    res={'fun':E,'x':params}    
    return res 
def Energy(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector() 
    E=1-numpy.abs(state_g@numpy.conjugate(state))**2#(numpy.conjugate(state)@Hmat@state).real
    if save_steps==True:
        with open('paramsForQubitAdapt_eg_model_parity_fidelity.txt','+a') as f:
            Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
            print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
    return E

In [4]:
def fun_jac(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector() 
    def State(params):
        circ=var_form_base.construct_circuit(parameters=params)
        state=execute(circ,backend,shots=1024).result().get_statevector() 
        #E=(numpy.conjugate(state)@Hmat@state).real
        return state
    setofParams1=[numpy.array(params)+numpy.array([0]*i+[numpy.pi/2]+[0]*(len(params)-i-1)) for i in range(len(params))]
    States=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(State)(setofParams1[i]) for i in range(len(params))))#qubit_pool_UCCSD))))#
    jac=[]
    for i in range(len(States)):
        jac.append((numpy.conjugate(state)@Hmat@States[i]).imag)
    return jac

In [5]:
U=7
#Constructing Hamiltonian
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)
#commutator pools
commutator_pool=commutatorPool(qubitH,stripZs=True)
#constructing matrices for gradient computation
MatrixRepOfPoolOps=list(map(commutatorsForGradient,commutator_pool))
print("pool sizes",len(MatrixRepOfPoolOps))
backend=Aer.get_backend('statevector_simulator')
#preparing HF state
circ=QuantumCircuit(6)
circ.x(0)
circ.x(3)
var_form_base=UCCSD(8,num_particles=4, initial_state=circ,qubit_mapping='parity',two_qubit_reduction=True)
var_form_base.manage_hopping_operators()
error=1000

16
pool sizes 16


In [6]:
save_steps=False
Energy([])

0.806210736697223

In [7]:
ExcOps=[]
params=[]
with open('paramsForQubitAdapt_eg_model_parity_fidelity.txt','r') as f:
    lines=f.readlines()
    for line in lines:
        if (line[0]=='l'):
            label=line[:-1]
            exc=label.split('-')[1][1:]
            print(exc)
            ExcOps.append(exc)
            var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli(exc)]]))
    params=eval(lines[-1])        

IIIIYX
IIIXYI
YXIIII
IYXIII
IIXYXX
XXYIIX
IIYIIX
IIXIIY
IIXYXX
IIIIYX
IIIYXI
IIIIXY
IXYIII
IYIIIX
YXXIIX
IIXIIY
IIYIIX
IIXYXX
IIXXXY
XXYIIX
XXYIIX
IIIIYX
IIIXYI
YXXIIX
IYIIIX
XXYIIX


In [8]:
Energy(params)

0.007544211834635939

In [9]:
len(params)

26

$|\Psi\rangle =\exp(i\theta_{1}A_{1})\exp(i\theta_{2}A_{2})\ldots \exp(i\theta_{n}A_{n})|0\rangle$

In [ ]:
#params=[]
EnergyArr=[]
#ExcOps=[]
ti=time.time()
steps=35
for i in range(35):
    save_steps=False
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector()
    grads=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(compute_gradient)(commutator_pool[i],state) for i in range(len(commutator_pool))))#qubit_pool_UCCSD))))#
    indexes=numpy.argsort(abs(grads))[::-1][:3]
    print("three highest grads",grads[indexes[0]],grads[indexes[1]],grads[indexes[2]])
    PauliOp=commutator_pool[indexes[0]]
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    print("chosen Op",ExcOps[-1])
    with open('paramsForQubitAdapt_eg_model_parity_fidelity.txt','a') as f:
        print("label-",ExcOps[-1],file=f)
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    #construct circuit for estimating Hamiltonian
    res=SMO(Energy,params,runs=10,tol=1e-6,save_opt_steps=True)
    E,params=res['fun'],list(res['x'])
    save_steps=True
    res=scipy.optimize.minimize(Energy,params,method='BFGS',options={'ftol':1e-7})
    params,E=list(res['x']),res['fun']
    print("num_params",var_form_base.num_parameters)
    print("Energy",E)
    EnergyArr.append(E)
    print("time elapsed",time.time()-ti)    
    error=EnergyArr[-1]-Eg

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  12 out of  16 | elapsed:    9.6s remaining:    3.2s
[Parallel(n_jobs=7)]: Done  16 out of  16 | elapsed:    9.8s finished


three highest grads 0.00012407616597182324 6.953833733912358e-05 5.453488753172838e-05
chosen Op IIIYXI
looped 1 times
inner loop error 0.00012804904132079908
looped 2 times
inner loop error 0.00016771231979761936
looped 3 times
inner loop error 0.00012955939081643608
looped 4 times
inner loop error 0.00011341635963302288
looped 5 times
inner loop error 0.00010283093455915049
looped 6 times
inner loop error 9.427078884199913e-05
looped 7 times
inner loop error 8.691134039989779e-05
looped 8 times
inner loop error 8.053130182711365e-05
looped 9 times
inner loop error 7.501556795125319e-05
num_params 27
Energy 0.004869191851124333
time elapsed 2576.7268357276917


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


three highest grads 9.83994272445281e-05 8.659652658975721e-05 5.085507986544901e-05
chosen Op IIXIIY
looped 1 times


[Parallel(n_jobs=7)]: Done  12 out of  16 | elapsed:    9.2s remaining:    3.1s
[Parallel(n_jobs=7)]: Done  16 out of  16 | elapsed:    9.2s finished


...

inner loop error 1.8349114076443573e-05
looped 2 times
inner loop error 2.3459500744227313e-05
looped 3 times
inner loop error 1.8287293983620945e-05
looped 4 times
inner loop error 1.5539345001924776e-05
looped 5 times
inner loop error 1.3519982206000591e-05
looped 6 times
inner loop error 1.2113269549307581e-05
looped 7 times
inner loop error 1.1151969051992339e-05
looped 8 times
inner loop error 1.0487055086383368e-05
looped 9 times
inner loop error 1.0012368096334878e-05
num_params 31
Energy 0.00014697835080523802
time elapsed 30618.007899284363


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


three highest grads 3.3454302301521245e-06 1.7224918644980708e-06 1.1447104438904865e-06
chosen Op IIXXXY
looped 1 times


[Parallel(n_jobs=7)]: Done  12 out of  16 | elapsed:    8.3s remaining:    2.8s
[Parallel(n_jobs=7)]: Done  16 out of  16 | elapsed:    8.3s finished


inner loop error 3.3440131094186754e-06
looped 2 times
inner loop error 4.023582378664958e-06
looped 3 times
inner loop error 2.683264381575512e-06
looped 4 times
inner loop error 2.1216030808846043e-06
looped 5 times
inner loop error 1.8156179372175174e-06
looped 6 times
inner loop error 1.619081042125714e-06
looped 7 times
inner loop error 1.4774067226275278e-06
looped 8 times
inner loop error 1.3668589268767128e-06
looped 9 times
inner loop error 1.27634180024927e-06


In [4]:
params_arr=[]
Labels_arr=[]
Energy_arr=[]
count=0
with open('paramsForQubitAdapt_eg_model_parity_fidelity.txt','r') as f:
    lines=f.readlines()
    for i in range(len(lines)):
        if lines[i][0]=='l':
            count=count+1
            label=lines[i].split('-')[1][1:-1]
            Labels_arr.append(label)
            if count>1:
                params,E=lines[i-1].split('#')
                params_arr.append(eval(params))
                Energy_arr.append(float(E))
    params_arr.append(eval(lines[-1]))
    Energy_arr.append(float(lines[-1].split('#')[1]))

In [5]:
params_arr=params_arr[0:32]
Energy_arr=Energy_arr[0:32]

In [6]:
for i in range(len(params_arr)):
    with open('data_eg_model_parity_fidelity.txt','+a') as f:
            print(Labels_arr[i]+'\n'+'['+','.join([str(params_arr[i][j]) for j in range(len(params_arr[i]))])+']'+'#'+"{:0.10f}".format(Energy_arr[i]),file=f)

In [7]:
len(Labels_arr)

32